# Brief analysis of the historical price of bitcoin

## Setup

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import signal
from statsmodels.tsa.seasonal import STL

In [ ]:
# Get bitcoin price df and set date as index, for more efficiency and simplicity
df_btc = pd.read_csv("../data/BTC.csv", index_col="date", parse_dates=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get OHLC average as price
df_btc["price"] = df_btc[["open", "high", "low", "close"]].mean(axis=1)
df_btc.drop(columns=["open", "high", "low", "close"], inplace=True)

## Dataset basic info

In [ ]:
# First and last entries
pd.concat([df_btc.head(1), df_btc.tail(1)]).T

In [ ]:
df_btc.describe().T

In [ ]:
# How many trading days per year on average
days_per_year = df_btc[~df_btc.index.year.isin([2024, 2010])].index.year.value_counts()
days_per_year.mean().round(2)

## Price of bitcoin across time

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_btc, x=df_btc.index, y="price", color="#f7931a", linewidth=0.75)

plt.title("Price of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

save_chart_as_png("1_BTC_price")

**It's much easier to see the early price fluctuations with a logarithmic scale on the y-axis**

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_btc, x=df_btc.index, y="price", color="#f7931a", linewidth=0.75)

plt.title("Price of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

plt.yscale("log")

save_chart_as_png("1_BTC_price_log")

In [ ]:
# All-time high
ath_date = df_btc["price"].idxmax()
df_btc.loc[[ath_date]]

In [ ]:
# All-time low
atl_date = df_btc["price"].idxmin()
df_btc.loc[[atl_date]]

In [ ]:
# Peaks
peaks, _ = signal.find_peaks(df_btc["price"], distance=365)
df_btc.iloc[peaks].nlargest(12, "price").sort_values("date")[["price"]].T

In [ ]:
# Valleys
valleys, _ = signal.find_peaks(-df_btc["price"], distance=300)
df_btc.iloc[valleys].nsmallest(12, "price").sort_values("date")[["price"]].T

In [ ]:
# Price appreciation since first entry
first_entry_price = df_btc.iloc[0]["price"]
last_entry_price = df_btc.iloc[-1]["price"]
(last_entry_price - first_entry_price) / first_entry_price

**Key takeaways:**
- ...

## STL decomposition (trend, seasonality, and residuals)

In [ ]:
stl = STL(df_btc["price"], period=365).fit()

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(10, 6), sharex=True)

axes[0].plot(df_btc.index, df_btc["price"], label="Original", color="#f7931a", linewidth=0.5)
axes[0].set_title("Price of bitcoin across time")

axes[1].plot(df_btc.index, stl.trend, label="Trend", color="aqua", linewidth=1)
axes[1].set_title("Trend component")

axes[2].plot(df_btc.index, stl.seasonal, label="Seasonal", color="fuchsia", linewidth=0.5)
axes[2].set_title("Seasonal component")

axes[3].plot(df_btc.index, stl.resid, label="Residual", color="red", linewidth=0.5)
axes[3].set_title("Residual component")

plt.tight_layout()

### Trend analysis

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(stl.trend, color="aqua", linewidth=1)

plt.title("Trend component of the price of bitcoin across time")
plt.xlabel("")
plt.ylabel("Trend")

plt.yscale("log")
plt.ylim(df_btc.min()["price"], plt.ylim()[1])

save_chart_as_png("1_BTC_trend")

**Key takeaways:**
- ...

### Seasonality analysis

In [ ]:
seasonal = stl.seasonal
monthly_avgs = seasonal.groupby(seasonal.index.month).mean()

plt.figure(figsize=(10, 6))

plt.plot(monthly_avgs.index, monthly_avgs.values, marker='o', color="fuchsia", linewidth=1)

plt.title("Average seasonal component of the price of bitcoin throughout the year")
plt.xlabel("Month")
plt.ylabel("Seasonality")

save_chart_as_png("1_BTC_seasonal")

**Key takeaways:**
- ...